# TODO

In [1]:
# Dans les arrêtés type "Agrément de direction", le titre de l'arrêté correspond à un visa, pas à l'agrément de direction

# @TODO ARRETE MODIFICATIF N° pour les arrêtés OCR
# Exemple: https://data.iledefrance.fr/pages/publication-des-actes-fiche-detail/?q=identifiant:%22AR2022-372%22

In [3]:
#@TODONettoyer signatures présidente en OCR <p>.{0,15}?</p><p>Valérie PÉCRESSE</p>

# Librairies

In [5]:
#Pour une déliberation
import platform
import os
import re
import fitz
from pdf2docx import parse
from typing import Tuple
from datetime import datetime
import pandas as pd
from tempfile import TemporaryDirectory, NamedTemporaryFile
from pathlib import Path
import pytesseract
from pytesseract import Output
from pdf2image import convert_from_path
from PIL import Image, ImageDraw
from unidecode import unidecode

# Pour arrêtés
import xml.etree.ElementTree as ET
from bs4 import BeautifulSoup
from lxml import html

# Pour contrôles open data
import requests
import json

from csv import DictWriter


# Fonctions

In [185]:
 if platform.system() == "Windows":
    #pytesseract.pytesseract.tesseract_cmd = (r"C:/Program Files/Tesseract-OCR/tesseract.exe")
    # Windows also needs poppler_exe
    path_to_poppler_exe = Path(r'C:\Users\utilisateur\Documents\Python Scripts\poppler-22.04.0\Library\bin')
    # Put our output files in a sane place...
    out_directory = Path(r"~\Desktop").expanduser()
else:
    out_directory = Path("~").expanduser() 

In [186]:
def convertir_date_ods(date: str) -> str:
    
    date = date.strip().rstrip("\r\n")
    
    # Convertir en YYYY-MM-DD
    annee = date[-4:]
    
    if (re.search(r"jan", date, re.IGNORECASE)):
        mois = "01"
    elif (re.search(r"vrier", date, re.IGNORECASE)):
        mois = "02"
    elif (re.search(r"mars", date, re.IGNORECASE)):
        mois = "03"
    elif (re.search(r"avril", date, re.IGNORECASE)):
        mois = "04"
    elif (re.search(r"juin", date, re.IGNORECASE)):
        mois = "06"
    elif (re.search(r"juil", date, re.IGNORECASE)):
        mois = "07"
    elif (re.search(r"ao", date, re.IGNORECASE)):
        mois = "08"
    elif (re.search(r"sep", date, re.IGNORECASE)):
        mois = "09"
    elif (re.search(r"oct", date, re.IGNORECASE)):
        mois = "10"
    elif (re.search(r"nov", date, re.IGNORECASE)):
        mois = "11"
    elif (re.search(r"cemb", date, re.IGNORECASE)):
        mois = "12"
    elif (re.search(r"mai", date, re.IGNORECASE)):
        mois = "05"
    else:
        mois = "08"
    
    jour = date[0:2].strip().rstrip("/-")
    if (len(jour) < 2):
        jour = "0" + jour
    
    date = annee + "-" + mois + "-" + jour
    return date

In [187]:
def parse_hocr_to_htmllist(hocr_string, num_page, filename, verbose: bool = False):
    
    identifiant = ""
    date_decision = ""
    date_controle = ""
    titre = ""
    titre_is_next = 0
    iteration_titre = 0
    date_decision_is_next = 0
    iteration_date = 0
    regex_titre = r"(?P<titre>(?:portant|de cession|apportant|de désignation|de design|de délég|de nominat|délégu|d'agrément|régissant|emportant|autoris|fixant|relatif|creant|concernant|créant|modifi).{10,150})"
    
    # <div class='ocr_page' id='page_1' title='image "C:\Users\UTILIS~1\AppData\Local\Temp\tess_j3a_34c0_input.JPEG"; bbox 0 0 4132 5848; ppageno 0'>
    #   <div class='ocr_carea' id='block_1_1' title="bbox 2781 116 3545 162">
    #     <p class='ocr_par' id='par_1_1' lang='fra' title="bbox 2781 116 3545 162">
    #       <span class='ocr_line' id='line_1_1' title="bbox 2781 116 3545 162; baseline 0.001 -10; x_size 47; x_descenders 10; x_ascenders 10">
    #          <span class='ocrx_word' id='word_1_1' title='bbox 2781 117 2937 162; x_wconf 96'>Envoyé</span>
    #          <span class='ocrx_word' id='word_1_2' title='bbox 2955 126 3004 153; x_wconf 96'>en</span>
    
    # <body>
    # <div class='ocr_page'
    # div class='ocr_carea'
    soup = BeautifulSoup(hocr_string, 'html.parser')
    
    paragraphes = []
    debut = False
    if (num_page > 0):
        debut = True
    fin = False
    
    for pages in soup.find_all("div", class_="ocr_page"):
        
        for paragraph in pages.find_all("p", class_="ocr_par"):
            #print("MON PARAGRAPH")
            
            text = ""
            for word in paragraph.find_all("span", class_="ocrx_word"):
                #print(word.get_text())
                #print(word.string)
                text += " " + word.get_text()
            
            if (verbose):
                print("===--TEXT--====")
                print(text)
                print("===----====")
            
            regex_controle_pref = r"Envoyé en préfecture le ([\d]{2})/([\d]{2})/([\d]{4})"
            detect_controle = re.findall(regex_controle_pref, text)
            if (len(detect_controle)):
                #date_controle
                #print(detect_controle)
                date_controle = "" + detect_controle[0][2] +"-" + detect_controle[0][1] +"-" + detect_controle[0][0]
            # Acte certifié exécutoire
            # Par publication ou notification le JJ/MM/AAAA
            # Par transmission au contrôle de Légalité le JJ/MM/AAAA
            regex_controle_pref2 = r"publication\s+ou\s+notification\s+le\s+([\d]{2})/([\d]{2})/([\d]{4})"
            regex_controle_pref3 = r"transmission\s+au\s+contrôle\s+de\s+Légalité\s+le\s+([\d]{2})/([\d]{2})/([\d]{4})"
            detect_controle2 = re.findall(regex_controle_pref2, text)
            detect_controle3 = re.findall(regex_controle_pref3, text)
            if (len(detect_controle2)):
                date_controle = "" + detect_controle2[0][2] +"-" + detect_controle2[0][1] +"-" + detect_controle2[0][0]
            if (len(detect_controle3)):
                date_controle = "" + detect_controle3[0][2] +"-" + detect_controle3[0][1] +"-" + detect_controle3[0][0]
            
            if (debut == True):
                
                if (verbose):
                    print("=DEBUT=")
                
                if (titre_is_next == 3):
                    # On ajoute au titre la chaîne
                    titre = (titre + " " + text.replace("/", "")).strip().rstrip('\r\n,.;')
                    titre_is_next = 2
                    
                cherche_titre = re.search(regex_titre, text.replace("/", ""), re.IGNORECASE)
                # On introduit la recherche du titre lorsque titre_is_next == 1
                if (iteration_titre < 3 and titre_is_next == 1):
                    if (cherche_titre is not None and len(cherche_titre.group('titre')) > 10 and titre == ""):
                        titre = cherche_titre.group('titre').strip().rstrip('\r\n,.;')
                        titre_is_next = 2
                        # Si titre tronqué, titre_is_next = 3
                        if (re.search('la Présidente du$', titre) is not None):
                            titre_is_next = 3
                    else:
                        iteration_titre += 1
                
                # On introduit la recherche de la date lorsque date_is_next == 1
                if (iteration_date < 3 and date_decision_is_next == 1 and text.replace("/", "").strip().rstrip('\r\n') != ""):
                    
                    print(" --- Date de décision prochaine")
                    regex_date_decision_seule = r"[DUdu\s]{2,4}(?P<jour>[\d]{1,2})[\s-]{1,2}(?P<mois>[jfmasondJFMASOND][\w]{2,9})[\s-]{1,2}(?P<annee>[\d]{4})"
                    
                    cherche_date_decision_seule = re.search(regex_date_decision_seule, text.replace("/", ""))
                    
                    if (cherche_date_decision_seule is not None):
                        #print(" - - - "+ str(cherche_date_decision_seule))
                        date_decision = convertir_date_ods(cherche_date_decision_seule["jour"] + " " + cherche_date_decision_seule["mois"] + " " + cherche_date_decision_seule["annee"])
                        date_decision_is_next = 2
                    else:
                        iteration_date += 1
            
            if (debut == False):
                #
                print("Texte replace / : " + text.replace("/", ""))
                # ARRETE N° 2020 - A6
                regex_complete_init_arrete = r"A[rêeténARETÊÉN° \s]{6,15}(?:[ MODIFCATmodifcatnN°\s]{10,17})?(?P<numero>(?:[\d]{4}|15|16|17|18|19|20)[-‐‑-−‒-‒––⁃─−―–−‒\s]{1,3}(?:[\d]{1,2}\s?[\d]{1,2})(?:[-‐‑-−‒-‒––⁃─−―–−‒\s]{1,3}[\d]{0,1})?)(?:[DUdu\s]{2,5}(?P<jour>[\d]{1,2})[\s]{1,3}(?P<mois>[A-zÀ-ú]{3,15})[\s]{1,3}(?P<annee>[\d]{4}))?(?:[\s\:]{0,4})?(?P<titre>(?:[\s]{0,2}[\w\’\'\"'-°àéèëêïöôüûù]{0,25}[\s]{0,2}[\d]{0,4}){1,20})?"
                regex_incomplete_arrete = r"A[rêeténARETÊÉN° \s]{6,15}(?:[ MODIFCATmodifcatnN°\s]{10,17})?(?P<numero>(?:[\d]{4}|15|16|17|18|19|20).{0,7})(?:\s|-)(?:[DUdu\s]{2,5}(?P<jour>[\d]{1,2})[\s]{1,3}(?P<mois>[A-zÀ-ú]{3,15})[\s](?P<annee>[\d]{4}))?(?:[\s\:]{0,4})?(?P<titre>(?:[\s]{0,2}[\w\’\'\"'-°àéèëêïöôüûù]{0,25}[\s]{0,2}[\d]{0,4}){1,20})?"
                
                detect_arrete = re.search(regex_complete_init_arrete, text.replace("/", ""))
                detect_arrete_incomplet = re.search(regex_incomplete_arrete, text.replace("/", ""))
                
                if (detect_arrete is not None):
                    
                    date_decision_is_next = 1
                    
                    print("Identifiant trouvé : " + text)
                    # [('2020- © }', '', '', '', "portant modification de l'arrêté n°16-154 du 30 mai 2016")]
                    
                    debut = True
                    
                    detect_arrete_dict = detect_arrete.groupdict()
                    print("Match : ", str(detect_arrete_dict))
                    
                    identifiant = detect_arrete_dict["numero"].replace(" ", "").rstrip('\r\n')
                    identifiant = prepare_identifiant_from_prepare(identifiant)
                    identifiant_compare = prepare_identifiant_from_prepare(filename)
                    if(identifiant != identifiant_compare):
                        identifiant = identifiant_compare
                        print("Identifiant non conforme au nom de fichier : ", identifiant, ". L'identifiant du nom de fichier a été récupéré : ", identifiant_compare)
                    
                    if (all(key in detect_arrete_dict for key in ("jour", "mois", "annee")) and detect_arrete_dict["jour"] is not None and detect_arrete_dict["mois"] is not None and detect_arrete_dict["annee"] is not None):
                        
                        date_decision_is_next = 2
                        
                        # [('2021-400', '17', 'décembre', '2021', 'RESTE')]
                        date_decision = convertir_date_ods(detect_arrete_dict["jour"] + " " + detect_arrete_dict["mois"] + " " + detect_arrete_dict["annee"]) # .replace(" ", "").rstrip('\r\n')
                        identifiant2 = detect_arrete_dict["numero"].replace(" ", "").rstrip('\r\n')
                        identifiant2 = prepare_identifiant_from_prepare(identifiant2)
                        if(identifiant2 != identifiant_compare):
                            identifiant2 = identifiant_compare
                        identifiant = identifiant2
                    
                    if ("titre" in detect_arrete_dict and len(detect_arrete_dict["titre"]) > 5):
                        titre = detect_arrete_dict["titre"].strip().rstrip('\r\n')
                        print("Titre trouvé dans la regex : ", titre)
                        titre_is_next = 2
                    else:
                        titre_is_next = 1
                    
                elif (detect_arrete_incomplet is not None):
                    
                    date_decision_is_next = 1
                    
                    print("Identifiant INCOMPLET trouvé : " + text)
                    # print(detect_arrete_incomplet)
                    debut = True
                    
                    detect_arrete_incomplet_dict = detect_arrete_incomplet.groupdict()
                    print("Match : ", str(detect_arrete_incomplet_dict))
                    
                    # Récupérer le nom de fichier
                    print("On utilise le nom de fichier : ", filename)
                    identifiant = prepare_identifiant_from_prepare(filename)
                    # print("Identifiant INCOMPLET trouvé : ", identifiant)
                    
                    if (all(key in detect_arrete_incomplet_dict for key in ("jour", "mois", "annee")) and detect_arrete_incomplet_dict["jour"] is not None and detect_arrete_incomplet_dict["mois"] is not None and detect_arrete_incomplet_dict["annee"] is not None):
                        
                        date_decision_is_next = 2
                        date_decision = convertir_date_ods(detect_arrete_incomplet_dict["jour"] + " " + detect_arrete_incomplet_dict["mois"] + " " + detect_arrete_incomplet_dict["annee"]) # .replace(" ", "").rstrip('\r\n')
                    
                    if ("titre" in detect_arrete_incomplet_dict and len(detect_arrete_incomplet_dict["titre"]) > 5):
                        titre = detect_arrete_incomplet_dict["titre"].strip().rstrip('\r\n')
                        print("Titre trouvé dans la regex : ", titre)
                        titre_is_next = 2
                    else:
                        titre_is_next = 1
                
                # @TODO inverser les deux blocs
                # if debut == True : ...
                # Puis if debut == False or ... (voir comment traiter le titre sur le paragraphe suivant l'identifiant c'est-à-dire quand debut == True)
                
            else :
                #if (date_non_trouvee):
                #    regex = r"([\d]{1,2}) ([A-zÀ-ú]{3,15}) ([\d]{4})"
                #    date = re.findall(regex, text)
                #    if (len(date)):
                #        date_decision = date[0][1] + " " + date[0][2] + " " + date[0][3]
                #        date_non_trouvee = False
                #print(text.strip())
                if (verbose):
                    print("=DEBUT=")
                
                if ("Envoyé en préfecture" in text):
                    continue
                elif (text == ""):
                    continue
                elif ("Reçu en préfecture le" in text):
                    continue
                elif ("Affiché le" in text):
                    continue
                elif ("ID :" in text):
                    continue
                else :
                    #print(text)
                    paragraphes.append("<p>" + text.strip() + "</p>")
    
    # result_set = soup.find_all(class_="ocr_page")
    #print(paragraphes)
    return titre, paragraphes, identifiant, date_decision, date_controle

In [188]:
def traitement_ocr(pdf_filename: str = ""):
    """Fonction de reconnaissance d'image, permet de lire le texte image
        Argument : le nom du fichier
        Retourne :- le texte contenue dans le pdf scanné ou non 
                  - la date de controle de légalité
    """
    image_file_list = []
    text_file = ""
    titre = ""
    paragraphes_html = []
    identifiant = ""
    date_decision = ""
    date_controle = ""
    
    with TemporaryDirectory() as tempdir:
        # Create a temporary directory to hold our temporary images.
        """
        Part #1 : Converting PDF to images
        """
        if platform.system() == "Windows":
            pdf_pages = convert_from_path(pdf_filename, 500, poppler_path=path_to_poppler_exe)
        else:
            pdf_pages = convert_from_path(pdf_filename, 500)
            # Read in the PDF file at 500 DPI
        # Iterate through all the pages stored above
        for page_enumeration, page in enumerate(pdf_pages, start=1):
            # Create a file name to store the image
            filename = f"{tempdir}\page_{page_enumeration:03}.jpg"
            # Declaring filename for each page of PDF as JPG
            page.save(filename, "JPEG")
            image_file_list.append(filename)
        """
        Part #2 - Recognizing text from the images using OCR
        """
        with NamedTemporaryFile() as output_file:
            # Iterate from 1 to total number of pages
            num_page = 0
            for image_file in image_file_list:

                tempimg = Image.open(image_file)
                #d = pytesseract.image_to_data(tempimg, lang="fra", output_type=Output.DICT, config="hocr")
                d = pytesseract.image_to_pdf_or_hocr(tempimg, lang='fra', extension='hocr').decode('utf-8')
                titre_part,liste_paragraphes,identifiant_part,date_decision_part,date_controle_part = parse_hocr_to_htmllist(d, num_page, pdf_filename)
                
                print("Identifiant part : " + identifiant_part)
                print("Date decision part : " + date_decision_part)

                paragraphes_html.append(liste_paragraphes)
                if (identifiant_part != ""):
                    identifiant = normalise_identifiant(identifiant_part)
                if (date_decision_part != ""):
                    date_decision = date_decision_part
                if (date_controle_part != ""):
                    date_controle = date_controle_part
                if (titre_part != ""):
                    titre = titre_part
                
                num_page += 1
                
                # At the end of the with .. output_file block
            # the file is closed after writing all the text.
        # At the end of the with .. tempdir block, the
        # TemporaryDirectory() we're using gets removed!       
    # End of main function!
    # print(paragraphes_html)
    return titre, text_file, paragraphes_html, identifiant, date_decision, date_controle


In [189]:
def date_publication(mise_en_ligne="ok"):
    if mise_en_ligne == 'ok':
        date_publication = datetime.today().strftime('%Y-%m-%d')
    else :
        date_publication = 'Non publiée'
    return date_publication

def numero_from_identifiant(identifiant: str) -> str:
    """TODO: à généraliser, utiliser la fonction des Avis CESER"""
    regex_identifiant = r"([\d]{4}[-‐‑-−‒-‒––⁃─−―–−‒\s]{1,3}[\d]{2,3}(?:[-‐‑-−‒-‒––⁃─−―–−‒\s]{1,3}[\d]{0,1})?)"
    cherche_identifiant = re.findall(regex_identifiant, identifiant)
    if (len(cherche_identifiant)):
        return "AR " + cherche_identifiant[0]
    return ""

def normalise_identifiant(identifiant: str) -> str:
    """Transforme un identifiant sous l'ancien format en identifiant d'arrêté avec AR"""
    regex_identifiant = r"(?P<annee>[\d]{4}|15|16|17|18|19|20)\s*[-‐‑-−‒-‒––⁃─−―–−‒\s]{1,3}\s*(?P<numero>[\d]{2,3})\s*(?:[-‐‑-−‒-‒––⁃─−―–−‒\s]{1,3}\s*(?P<complement>\d))?"
    cherche_identifiant = re.search(regex_identifiant, identifiant)
    if (cherche_identifiant is not None):
        ientifiant = "AR" + cherche_identifiant.group('annee') + "-" + cherche_identifiant.group('numero')
        if (cherche_identifiant.group('complement') is not None):
            identifiant = identifiant + "-" + cherche_identifiant.group('complement')
        return identifiant
    return ""

def append_dict_as_row(file_name, dict_of_elem, field_names):
    # Open file in append mode
    with open(file_name, 'a+', newline='', encoding="utf-8") as write_obj:
        # Create a writer object from csv module
        dict_writer = DictWriter(write_obj, fieldnames=field_names)
        # Add dictionary as wor in the csv
        #print("=-=-=-", dict_of_elem)
        dict_writer.writerow(dict_of_elem)
    return

def replacement_numero_article(text: str, verbose: bool = False) -> str:
    """Fonction qui retourne le texte modifié sur tous les numéros d'articles"""
    
    # Chercher tous les articles X et regarder ensuite si le match contient immédiatement avant, et immédiatement après un <p>
    # Dans la regex ci-dessous, les (.{5})? contiennent les 5 caractères immédiatement avant et immédiatement après, s'il y en a u moins 5 avant et après
    regex_article_semantique = r"(?P<avant>.{5})?[\s]*(?P<bouvrant><b>)?[\s]*A(?:rticle|RTICLE)[\s]*(?P<numero>[\d]+)(?:er|°)?[\s]*[:]?[\s]*(?P<bfermant><\/b>)?[\s]*(?P<apres>.{5})?"
    
    # @TOD0 prévoir l'absence de : ou une balise <b> fermant avant le :
    
    def re_replacement_numero_article(match: re.Match) -> str:
        """Fonction à utiliser comme paramètre replacement de la fonction re.sub pour nettoyer les numéros d'articles"""
        
        bouvrant = "<b>"
        bfermant = "</b>"
        
        dictionnaire = match.groupdict()
        # Contains {'first_name': 'Malcolm', 'last_name': 'Reynolds'}
        new_text = ""
        if verbose: print("Nouveau match : ",str(dictionnaire))
        if dictionnaire["avant"] is not None:
            # Voir si contient <p>
            new_text += dictionnaire["avant"]
            if "<p>" in dictionnaire["avant"]:
                if verbose: print("Précédé d'un début de paragraphe: ", dictionnaire["avant"])
            else:
                if verbose: print("Non précédé d'un début de paragraphe: ", dictionnaire["avant"])
                if dictionnaire["bouvrant"] is None and dictionnaire["bfermant"] is not None: new_text += bfermant
                new_text += "</p><p>"

        # Traitement des balises bold b
        if dictionnaire["bouvrant"] is not None and dictionnaire["bfermant"] is not None:
            if verbose: print("On avait du bold ouvrant et fermant, on ne fait rien")
        if dictionnaire["bouvrant"] is not None and dictionnaire["bfermant"] is None:
            if verbose: print("On avait du bold ouvrant mais PAS de fermant, on ne doit pas ajouter le b fermant")
            bfermant = ""

        if dictionnaire["numero"] is not None: # le contraire ne devrait jamais arriver
            new_text += bouvrant + "Article " + dictionnaire["numero"] + "&nbsp;:" + bfermant + " "
        else:
            print("Erreur, la regex ne devrait pas prendre des articles sans numéro")
        if dictionnaire["apres"] is not None:
            # Voir si contient </p>? Finalement, non car peut contenir titre d'article
            # On se contente de ne pas supprimer la partie après
            new_text += dictionnaire["apres"]

        if verbose: print("New text : ", new_text)

        return new_text
    
    text = re.sub(regex_article_semantique, re_replacement_numero_article, text)
    
    return text

def clean_html_arrete(text: str, verbose: bool = False) -> str:
    """Fonction permettant de nettoyer le code html d'un bloc via des fonctions génériques"""
    
    #print("---------------------")
    #print("-------DEBUT---------")
    #print(text)
    
    # Nettoyer les balises inutiles; exemples </b>[\s]+<b> OU <b>[\s]+</b>
    # 1ère: bold interrompu
    regex_bold_interrompu = r"<\/b>\s*<b>" # à remplacer par une espace
    text = re.sub(regex_bold_interrompu, " ", text)
    # 2ème: bold vide
    regex_bold_vide = r"<b>\s*<\/b>" # à remplacer par une espace
    text = re.sub(regex_bold_vide, " ", text)
    # 2ème: espace entre balises p (3 étapes)
    regex_balises_p_separees = r"</p>\s+<p>"
    text = re.sub(regex_balises_p_separees, "</p><p>", text)
    regex_balises_p_separees = r"<p>\s+<"
    text = re.sub(regex_balises_p_separees, "<p><", text)
    regex_balises_p_separees = r">\s+<\/p>"
    text = re.sub(regex_balises_p_separees, "></p>", text)
    
    # Ajouter des fins de paragraphe avant et après les numéros d'articles:  <b>ARTICLE 1 : </b>
    text = replacement_numero_article(text, verbose)
    
    # @TODO Ajouter le cas où un <b> "dépasse" avant ou après le numéro d'article. Voir arrêté 2022-296 et 2022-363
    # RÊTÉ </b></p><p> <b>ARTICLE 1 : </b> <b> </b> <b>Monsieur Patrick KARAM</b> ,
    # erezerfs. </p><p>   <b>Article 4        :      dispositifs in
    
    # Voir pour tests: https://regex101.com/r/sLFDgf/1
    
    # @TODO nettoyer les en tetes
    # <p> <b>Conseil régional </b> <b> </b> 2, rue Simone Veil – 93400 Saint-Ouen-sur-Seine  Tel: 01 53 85 53 85 –  www.iledefrance.fr            RegionIleDeFrance            @iledefrance     </p>
    # <p> <b>Région Île-de-France </b> <b> </b></p><p> 2, rue Simone Veil – 93400 Saint-Ouen  Tél. : 01 53 85 53 85 –  www.iledefrance.fr  </p>
    
    # 9ème: espaces multiples
    regex_espaces_multiples = r"\s{2,}" # à remplacer par une espace
    text = re.sub(regex_espaces_multiples, " ", text)
    # 9ème: blocs vides (return "")
    regex_bloc_vide = r"\s*<p>\s*<\/p>\s*"
    text = re.sub(regex_bloc_vide, "", text)
    
    regex_balises_p_separees = r"<p>\s+<"
    text = re.sub(regex_balises_p_separees, "<p><", text)
    regex_balises_p_separees = r">\s+<\/p>"
    text = re.sub(regex_balises_p_separees, "></p>", text)
    
    # <p><b>Conseil régional</b></p>
    regex_blocs_a_supprimer = r"<p><b>Conseil régional<\/b><\/p>"
    text = re.sub(regex_blocs_a_supprimer, "", text, flags=re.IGNORECASE)
    # <p> 2, rue Simone Veil – 93400 Saint-Ouen-sur-Seine Tél : 01 53 85 53 85 – www.iledefrance.fr</p>
    regex_blocs_a_supprimer = r"<p>[\s — -,:\.]*2[\s — -,:\.]*rue Simone Veil[\s — -,:\.]*93400[\s — -,:\.]*Saint-Ouen(-sur-Seine)?[\s — -,:\.]*(Tél|Tel|Téléphone)?[\s — -,:\.]*((\d\s){3}){5}[\s — -,:\.]*www.iledefrance.fr[\s — -,:\.]*(RegionIleDeFrance)?[\s — -,:\.]*(@iledefrance)?[\s -,:\.]*<\/p>"
    text = re.sub(regex_blocs_a_supprimer, "", text, flags=re.IGNORECASE)
    # <p> RegionIleDeFrance @iledefrance</p>
    regex_blocs_a_supprimer = r"<p>[\s -,:\.]*RegionIleDeFrance[\s -,:\.]*@iledefrance[\s -,:\.]*<\/p>"
    text = re.sub(regex_blocs_a_supprimer, "", text, flags=re.IGNORECASE)
    
    # <p>Conseil régional</p>.{0,5}<p>.{0,5}2.{0,5}rue Simone Veil.{0,5}93400.{0,5}SAINT-OUEN.{0,5}T(?:é|e)l.{0,5}01.?53.?85.?53.?85.{0,5}Fax.{0,5}01.?53.?85.?53.?89.?www.?iledefrance.?fr.{0,5}</p>.{0,5}<p>.{0,50}</p>
    regex_blocs_a_supprimer = r"<p>Conseil régional(?:</p>.{0,5}<p>|.{0,10}).{0,5}2.{0,5}rue Simone Veil.{0,10}93400.{0,10}SAINT-OUEN.{0,10}T(?:é|e)l.{0,10}01.{0,2}53.{0,2}85.{0,2}53.{0,2}85.{0,10}(?:Fax.{0,10}01.{0,2}53.{0,2}85.{0,2}53.{0,2}89)?.?www.?iledefrance.?fr.{0,5}</p>(?:[^<>]){0,10}(?:<p>.{0,50}</p>)?"
    text = re.sub(regex_blocs_a_supprimer, "", text, flags=re.IGNORECASE)
    
    # <p>.{0,10}Région Île-de-France.{0,10}</p>.{0,10}<p>2.{0,5}rue Simone Veil.{0,10}93400.{0,5}Saint-Ouen.{0,10}T(?:é|e)l.{0,5}01.?53.?85.?53.?85.{0,10}www.iledefrance.fr</p>.{0,10}<p>R.{0,10}leDeFrance.{0,10}@.{0,10}de.?rance</p>
    regex_blocs_a_supprimer = r"<p>.{0,10}Région Île-de-France.{0,10}</p>(?:[^<>]){0,10}<p>2.{0,5}rue Simone Veil.{0,10}93400.{0,5}Saint-Ouen.{0,10}T(?:é|e)l.{0,5}01.?53.?85.?53.?85.{0,10}www.iledefrance.fr</p>(?:[^<>]){0,10}<p>R.{0,10}leDeFrance.{0,10}@.{0,10}de.?rance</p>"
    text = re.sub(regex_blocs_a_supprimer, "", text, flags=re.IGNORECASE)
    
    regex_signature_a_nettoyer = r"<p>.{0,13}?</p><p>Valérie P(?:É|E)CRESSE</p>"
    text = re.sub(regex_signature_a_nettoyer, "<p>Valérie PÉCRESSE</p>", text)
    
    # article 1°[\S]{0,3}
    regex_article_premier = r"article 1°[\S]{0,3}"
    text = re.sub(regex_signature_a_nettoyer, "article 1er", text)
    
    # re.findall(regex_article_semantique, block_text)
    # re.match(regex_article_semantique, block_text)
    # Match.expand(template)
    # Return the string obtained by doing backslash substitution on the template string template,
    # as done by the sub() method. Escapes such as \n are converted to the appropriate characters,
    # and numeric backreferences (\1, \2) and named backreferences (\g<1>, \g<name>) are replaced
    # by the contents of the corresponding group.
    # re.sub(pattern, replacement, string) replaces all occurrences of pattern by replacement in string
    # C'est un negative lookahead pour vérifier si la string est suivie par... (?!...) 
    # et un negative lookbehind pour vérifier si la string est précédée par ... (?<!...)
    
    #print("--------FIN----------")
    #print(text)
    #print("---------------------")
    #print("---------------------")
    
    return text

def prepare_identifiant_from_prepare(identifiant: str) -> str:
    """Nettoie l'identifiant à partir d'un identifiant préparé, c'est-à-dire extrait par une regex large spectre"""
    regex_identifiant_prepare = r"(?P<annee>[\d]{4}|15|16|17|18|19|20)[-‐‑-−‒-‒––⁃─−―–−‒\s]{1,3}(?P<numero>[\d\s]{2,4})(?:[-‐‑-−‒-‒––⁃─−―–−‒\s]{1,3}(?P<supplement>[\d]{0,1}))?"
    identifiant_prepare = re.search(regex_identifiant_prepare, identifiant, re.IGNORECASE)
    if identifiant_prepare:
        annee = identifiant_prepare.group('annee')
        numero = identifiant_prepare.group('numero').replace(" ","")
        if(len(numero) == 2):
            numero = "0" + numero
        supplement = identifiant_prepare.group('supplement')
        identifiant = "AR" + annee + "-" + numero
        if supplement is not None and len(supplement) > 0:
            identifiant += "-" + str(supplement)
    else:
        print("Identifiant non reconnu par regex : ", identifiant)
    return identifiant

In [190]:
def dict_to_html(dictionary_elements: dict, verbose: bool = False) -> str:
    block_text = ""
    for block in dictionary_elements['blocks']:
        #print(block)
        
        # TODO Pour les visas, préférer les ul / li aux paragraphes classiques
        
        block_text += "<p>"
        for line in block['lines']:
            line_text = ""
            for span in line['spans']:
                #print(span)
                if "Bold" in span['font'] or "bold" in span['font']:
                    if "Italic" in span['font'] or "italic" in span['font']:
                        texte_a_inclure = "<b><i>" + span['text'] + "</i></b>"
                    else:
                        texte_a_inclure = "<b>" + span['text'] + "</b>"
                else :
                    texte_a_inclure = span['text']
                line_text += " " + texte_a_inclure
                #print(span)
            block_text += line_text
            #line_text += " STOP "
            #print(line_text)
        #block_text = re.sub(' +', ' ', block_text.strip())
        block_text += "</p>"
        
    # On nettoie le code avec une fonction personnalisée
    block_text = clean_html_arrete(block_text, verbose)
        
    return block_text

def nettoie_date_aaaa_mois_jj(date):
    regex_date_francais = r"(?P<annee>[\d]{4})-(?P<mois>[jfmasondJFMASOND][\w]{2,9})-(?P<jour>[\d]{1,2})"
    match_date_decision = re.search(regex_date_francais, date, re.IGNORECASE)
    if match_date_decision:
        mois = match_date_decision.group('mois')
        if (re.search(r"jan", mois, re.IGNORECASE)):
            mois = "01"
        elif (re.search(r"vrier", mois, re.IGNORECASE)):
            mois = "02"
        elif (re.search(r"mars", mois, re.IGNORECASE)):
            mois = "03"
        elif (re.search(r"avril", mois, re.IGNORECASE)):
            mois = "04"
        elif (re.search(r"juin", mois, re.IGNORECASE)):
            mois = "06"
        elif (re.search(r"juil", mois, re.IGNORECASE)):
            mois = "07"
        elif (re.search(r"ao", mois, re.IGNORECASE)):
            mois = "08"
        elif (re.search(r"sep", mois, re.IGNORECASE)):
            mois = "09"
        elif (re.search(r"oct", mois, re.IGNORECASE)):
            mois = "10"
        elif (re.search(r"nov", mois, re.IGNORECASE)):
            mois = "11"
        elif (re.search(r"cemb", mois, re.IGNORECASE)):
            mois = "12"
        elif (re.search(r"mai", mois, re.IGNORECASE)):
            mois = "05"
        else:
            mois = "08"
        jour = "0" + match_date_decision.group('jour')
        jour = jour[-2:]
        return match_date_decision.group('annee') + "-" + mois + "-" + jour
    else:
        return ""


In [191]:
def extract_all_arrete(dictionary_elements: dict, verbose: bool = False):
    
    limite_inf = 0
    limite_sup = 950
    corps_visa_arrete = ""
    controle_legalite = ""
    date_decision = ""
    date_decision_debut = False
    fin = False
    
    dictionnaire_expurge = {'blocks': []}
    
    for block in dictionary_elements['blocks']:

        x0, y0, x1, y1 = block['bbox']
        
        texte = ""
        
        for line in block['lines']:
            for span in line['spans']:
                texte += " " + span['text']
        
        if (y0 > limite_inf and y1 < limite_sup):
            dictionnaire_expurge['blocks'].append(block)
        else:
            print("SUPPRIME en-tête / pied de page : " + texte)
            
        if ("Envoyé en préfecture le" in texte):
            # On enregistre le contrôle de légalité
            controle_legalite = texte
            #fin = True
            break
                
        if ("Fait à" in texte):
            date_decision_debut = True
        
        if (date_decision_debut and date_decision == ""):
            regex_date_francais = r"(?P<jour>[\d]{1,2})[\s-]{1,5}(?P<mois>[JFMASONDjfmasond][\w]{2,9})[\s-]{1,5}(?P<annee>[\d]{4})"
            match_date_decision = re.search(regex_date_francais, texte, re.IGNORECASE)
            if match_date_decision:
                date_decision = nettoie_date_aaaa_mois_jj(match_date_decision.group('annee') + "-" + match_date_decision.group('mois') + "-" + match_date_decision.group('jour'))
        
        #Fait à Saint-Ouen-sur-Seine en 1 exemplaire
        #Le
        #La présidente du conseil régional
        #Valérie PECRESSE
        #Par délégation,
        #L’adjointe à la cheffe du service des relations
        #avec les organismes
        #Laurence LARIDANT
        #09 Mai 2022
            
        """
        CONTROLE SUR LES SUPPRESSIONS LIEES AUX MARGES
        if (y0 > limite_inf and y1 < limite_sup):
            print("y0 = " + str(y0) + " et y1 = " + str(y1) + " :" + texte)
        else:
            print(" ")
            print("Supprimé: " + texte)
            print(" ")
        """
    corps_visa_arrete = dict_to_html(dictionnaire_expurge, verbose)

    return (corps_visa_arrete, controle_legalite, date_decision, fin)

In [192]:
def extract_id_titre_date_arrete(dictionary_elements: dict, page_annots = None, verbose: bool = False):
    ide = ""
    titre = ""
    identifiant_complet = False
    identifiant_tronque_a_completer = ""
    yinitial = 0
    titre_is_next = 0
    iteration_titre = 0
    date_decision_is_next = 0
    iteration_date = 0
    date_decision = ""
    
    font = ""
    size = ""
    font_titre = ""
    size_titre = ""
    
    regex_identifiant_arrete_complet = r"[arêeténARETÊÉN° ]{6,15}(?:[ MODIFCATRmodifcatnN°\s]{10,17})?(?P<identifiant>[\d]{4}\s?-\s?[\d]{2,3}(?:-[\d]{0,1})?)(?:[\s][dDuU]{2}[\s](?P<jour>[\d]{1,2})[\s-]{1,2}(?P<mois>[jfmasondJFMASOND][\w]{2,9})[\s-]{1,2}(?P<annee>[\d]{4}))" # Ex: 2022-342-1
    regex_identifiant_arrete_numsansdate = r"[arêeténARETÊÉN-°\s]{6,15}(?:[ MODIFCATRmodifcatnN°\s]{10,17})?(?P<identifiant>[\d]{4}\s*-\s*[\d]{2,3}(?:\s?-\s?[\d]{0,1})?)" # Juste 2022-123
    regex_identifiant_arrete_tronque = r"[arêeténARETÊÉN-°\s]{6,15}(?:[ MODIFCATRmodifcatnN°\s]{10,17})?([\d]{4}\s?-)" # Juste 2022-
    regex_id_controle_legalite = r"(?:ID :)[\s]{0,3}[\d]{2,3}-[\d]{5,20}-[\d]{5,20}-([\d]{4})_([\d]{3})[_]?([\d]{1})?-AR"
    regex_trois_chiffres = r"^[\s]{0,5}([\d]{3})[\s]{0,5}$"
    regex_date_decision_seule = r"^\s*[DUdu\s]{2,4}(?P<jour>[\d]{1,2})[\s-]{1,2}(?P<mois>[jfmasondJFMASOND][\w]{2,9})[\s-]{1,2}(?P<annee>[\d]{4})"
    regex_date_decision_annot = r"[DUdu\s]{2,4}(?P<jour>[\d]{1,2})[\s-]{1,2}(?P<mois>[jfmasondJFMASOND][\w]{2,9})[\s-]{1,2}(?P<annee>[\d]{4})"
    regex_titre = r"(?P<titre>(?:portant|de cession|apportant|de désignation|de design|de délég|de nominat|délégu|d'agrément|régissant|emportant|autoris|fixant|relatif|creant|concernant|créant|modifi).{10,150})"
    
    for annot in page_annots:
        if ("FreeText" in annot.type and annot.info is not None):
            cherche_identifiant_sansdate = re.search(regex_identifiant_arrete_numsansdate, annot.info['content'])
            if (cherche_identifiant_sansdate is not None):
                ide = "AR" + cherche_identifiant_sansdate.group('identifiant').replace(" ", "").rstrip('\r\n')
                if (verbose): print("IDE récupéré dans une annotation", ide)
            cherche_date_decision_annot = re.search(regex_date_decision_annot, annot.info['content'])
            if (cherche_date_decision_annot is not None):
                date_decision = convertir_date_ods(cherche_date_decision_annot.group('jour') + " " + cherche_date_decision_annot.group('mois') + " " + cherche_date_decision_annot.group('annee'))
                if (verbose): print("DATE DECISION récupérée dans une annotation", date_decision)
            # print(annot.info)
        elif ("Stamp" in annot.type):
            if (verbose): print(annot.get_text())
        else:
            if (verbose): print("Autre type : ", annot.type)
    
    for block in dictionary_elements['blocks']:
        
        # block_text = ""
        #print("BLOCK")
        #print(block)
        
        for line in block['lines']:
            line_text = ''
            for span in line['spans']:
                line_text += " " + span['text']
                font = str(span['font'])
                size = str(span['size'])[:5]
            
            # block_text += " " + line_text.strip().rstrip('\r\n')
            
            cherche_identifiant_complet = re.findall(regex_identifiant_arrete_complet, line_text)
            cherche_identifiant_numsansdate = re.findall(regex_identifiant_arrete_numsansdate, line_text)
            cherche_identifiant_tronque = re.findall(regex_identifiant_arrete_tronque, line_text)
            cherche_id_controle_legalite = re.findall(regex_id_controle_legalite, line_text)
            cherche_trois_chiffres = re.findall(regex_trois_chiffres, line_text)
            
            cherche_titre = re.search(regex_titre, line_text, re.IGNORECASE)
            
            if (titre_is_next == 2):
                # On compare la taille et la font avec celle du titre
                if (font == font_titre and size == size_titre):
                    # On ajoute la ligne au titre
                    titre = titre + " " + line_text.strip()
                titre_is_next = 3
            
            if (iteration_titre < 3 and titre_is_next == 1 and cherche_titre is not None and len(cherche_titre.group('titre')) > 5 and titre == ""):
                # On stocke la font + la taille de police pour repérer le prochain bloc
                font_titre = font
                size_titre = size
                titre = cherche_titre.group('titre').strip().rstrip('\r\n')
                # print("TITRE ARRIVE ICI : " + line_text + " // " + cherche_titre.group('titre'))
                titre_is_next = 2
            else:
                iteration_titre += 1
            
            if (date_decision_is_next == 1 and line_text.strip().rstrip('\r\n') != ""):
                if (verbose): print(" --- Date de décision prochaine")
                cherche_date_decision_seule = re.findall(regex_date_decision_seule, line_text)
                #print(" ICI ")
                #print(cherche_date_decision_seule)
                # print(" ICI ")
                # print(cherche_titre)
                if (cherche_titre is not None and len(cherche_titre.group('titre')) and titre == ""):
                    font_titre = font
                    size_titre = size
                    titre = cherche_titre.group('titre')
                    # print(" TITRE FIXE ICI : " + titre)
                    titre_is_next = 2
                if (len(cherche_date_decision_seule) and len(cherche_date_decision_seule[0]) > 2):
                    #print(" - - - "+ str(cherche_date_decision_seule))
                    date_decision = convertir_date_ods(cherche_date_decision_seule[0][0] + " " + cherche_date_decision_seule[0][1] + " " + cherche_date_decision_seule[0][2])
                    date_decision_is_next = 2
                    if (titre_is_next < 2):
                        titre_is_next = 1
            
            if (len(cherche_identifiant_complet) and identifiant_complet == False and identifiant_tronque_a_completer == "" and ide == ""):
                if (verbose): print(" --- IDE COMPLET")
                if (verbose): print(cherche_identifiant_complet)
                # [('2022-209', '7', 'juillet', '2022')]
                ide = "AR" + cherche_identifiant_complet[0][0].replace(" ", "").rstrip('\r\n')
                if (len(cherche_identifiant_complet[0]) > 2):
                    if (verbose): print("DATE TROUVEE AU DEBUT")
                    date_decision = convertir_date_ods(cherche_identifiant_complet[0][1] + " " + cherche_identifiant_complet[0][2] + " " + cherche_identifiant_complet[0][3]) #TODO
                identifiant_complet = True
                titre_is_next = 1
            elif (identifiant_complet == False and ide == ""):
                # Prendre en compte les numéros d'arrêtés qui ont été ajoutés à posteriori, avant envoi au contrôle de légalité
                # print("pas trouvé // " + line_text)
                if (len(cherche_identifiant_numsansdate) and identifiant_tronque_a_completer == ""):
                    # print(" --- On a trouvé le numéro ide sans date")
                    # On a trouvé le numéro ide sans date
                    date_decision_is_next = 1
                    ide = "AR" + cherche_identifiant_numsansdate[0]
                elif (identifiant_tronque_a_completer != ""):
                    # On a trouvé et enregistré la partie tronquée de l'identifiant tronqué
                    # print("ID TRONQUE DEJA COMPLETE // " + identifiant_tronque_a_completer)
                    # print("LINE " + line_text)
                    if(len(cherche_trois_chiffres)): # Trouve complément
                        # print("TROIS CHIFFRES TROUVES //" + line_text)
                        a, yfinal, c, d = line["bbox"]
                        # print("Ecart des y0 : init " + str(yinitial) + " - yfin " + str(yfinal) + " = " + str(yinitial-yfinal))
                        if (abs(yinitial-yfinal) > 10):
                            print("Alerte écart important : " + str(yinitial-yfinal))
                        ide = "AR" + identifiant_tronque_a_completer + cherche_trois_chiffres[0]
                        print("Identifiant complet " + ide)
                if (len(cherche_identifiant_tronque) and identifiant_tronque_a_completer == "" and ide == ""):
                    if (verbose): print("ID TRONQUE TROUVE // " + line_text)
                    #On a trouvé un identifiant tronqué à compléter
                    identifiant_tronque_a_completer = cherche_identifiant_tronque[0].replace(" ", "").rstrip('\r\n')
                    #On enregistre la hauteur y0 de la ligne
                    a, yinitial, c, d = line["bbox"]
                    titre_is_next = 1
        
        # print("BLOCK : " + block_text.strip().rstrip('\r\n'))      
    
    ide = prepare_identifiant_from_prepare(ide)
    #if (titre_is_next >= 2):
    #    titre = "Arrêté n°" + numero_from_identifiant(ide) + " " + titre
    #else:
    #    titre = "Arrêté n°" + numero_from_identifiant(ide)
    #print ("TITRE : " + titre)
        
    return (ide, titre.strip(), date_decision)

In [193]:
def extract_date_controle(controle_legalite: str) -> str:
    # regex_controle = r"Envoyé en préfecture le[\s]{0,3}([\d]{2})/([\d]{2})/([\d]{4})" #13/05/2022
    regex_controle = r"le\s{0,3}([\d]{2})\s*/\s*([\d]{2})\s*/\s*([\d]{4})" #13/05/2022
    date = re.findall(regex_controle, controle_legalite)
    if (len(date)):
        annee = date[0][2]
        mois = date[0][1]
        jour = date[0][0]
        return annee + "-" + mois + "-" + jour
    else:
        return ""
    

# Traitement

In [202]:

def traite_fichier_arrete(nom_fichier: str, verbose: bool = False) -> dict:
    """Analyse un fichier"""
    
    # Initialiser les valeurs
    identifiant = ""
    numero = ""
    titre = ""
    controle_legalite = ""
    date_decision = ""
    corps_visa_arrete = ""
    url1 = ""
    
    pdf = fitz.open(nom_fichier)
    
    #Est-ce un rapport ou une délibération ?
    page_de_garde = pdf.load_page(0).get_text()
    
    #Extraire contrôle de légalité
    controle_legalite = extract_date_controle(page_de_garde)
    
    # print(page_de_garde)
    detect_arrete = re.findall(r"[arêeténARETÊÉN° ]{6,15}(?:[ MODIFCATmodifcatnN°\s]{10,17})?([\d]{4}-[\d]{2,3}(?:-[\d]{0,1})?)", page_de_garde)
    if (len(detect_arrete) and len(page_de_garde) > 200):
        #if ("ARRETÉ N°" in page_de_garde or "ARRÊTÉ N°" in page_de_garde):
        type_document = "arretetext"
    elif (("Envoyé en préfecture le" in page_de_garde) or ("Reçu en préfecture le" in page_de_garde)):
        type_document = "arreteocr"
    else:
        type_document = "arreteocr"
    
    # Contrôles de cohérence et message
    messages = []
    if ((type_document == "arretetext" or type_document == "arreteocr") and not "AR" in nom_fichier):
        messages.append("AR non présent dans le nom de fichier")
    if ("RAP" in nom_fichier or "DEL" in nom_fichier):
        messages.append("RAP ou DEL présent dans le nom de fichier")
    if (type_document == "autre"):
        messages.append("Fichier non reconnu: " + str(pdf))
    
    #
    # Analyse des arrêtes sans OCR
    #
    if (type_document == "arretetext"):
        
        print(" =====TEXT===== ")
        print("DEBUT DOC " + nom_fichier)
        
        # Récupérer identifiant, titre (et éventuellement date si présente)
        page = pdf.load_page(0)
        tp = page.get_textpage()
        dictionary_elements = tp.extractDICT()
        for annot in page.annots():
            if ("FreeText" in annot.type):
                if (verbose): print("Annotation de type FreeText : ", annot.info)
            elif ("Stamp" in annot.type):
                if (verbose): print("Annotation de type Stamp : ", annot.get_text())
            else:
                if (verbose): print("Autre type d'annotation : ", annot.type)
        identifiant, titre, date_decision_part_debut = extract_id_titre_date_arrete(dictionary_elements, page.annots(), verbose)
        
        identifiant_compare = prepare_identifiant_from_prepare(nom_fichier)
        if (identifiant != identifiant_compare):
            identifiant = identifiant_compare
            print("Identifiant non conforme au nom de fichier : ", identifiant, ". L'identifiant du nom de fichier a été récupéré : ", identifiant_compare)

        # OK print(identifiant)
        # OK print(titre)
        
        # print("DATE DECISION RECUE PART DEBUT : " + date_decision_part_debut)
        
        numero = numero_from_identifiant(identifiant)
        url1 = "https://www.iledefrance.fr/actes/arretes/" + nom_fichier
        
        # Récupérer corps_visa_delib
        corps_visa_arrete = ""
        
        for i in range(0, pdf.page_count):
            page = pdf.load_page(i)
            tp = page.get_textpage()
            dictionary_elements = tp.extractDICT()
            corps_visa_arrete_part, controle_legalite_part, date_decision, fin = extract_all_arrete(dictionary_elements, verbose)
            corps_visa_arrete += corps_visa_arrete_part
            controle_legalite += controle_legalite_part
            # print("DATE DECISION RECUE : " + date_decision)
            if (fin):
                break
            
        if (controle_legalite != ""):
            print("Controle " + controle_legalite)
            controle_legalite = extract_date_controle(controle_legalite)
        
        if (date_decision == ""):
            print("Date_decision vide, on prend date_decision_part_debut : " + date_decision_part_debut)
            date_decision = date_decision_part_debut
    
    #
    # Analyse des arrêtes avec OCR
    #
    if (type_document == "arreteocr"):
        
        print(" =====OCR===== ")
        print("DEBUT DOC " + nom_fichier)
        
        identifiant = nom_fichier
        titre_part, text, paragraphes_html,identifiant,date_decision,date_controle = traitement_ocr(nom_fichier)
        
        if (titre_part != ""):
            titre += " " + titre_part
        titre = titre.replace("  ", " ").strip()
        # print("=======",paragraphes_html)
        for blocks in paragraphes_html:
            for paragraphes in blocks:
                #print(paragraphes_html)
                corps_visa_arrete += paragraphes
        
        numero = numero_from_identifiant(identifiant)
        url1 = "https://www.iledefrance.fr/actes/arretes/" + nom_fichier
        
        if (controle_legalite == ""):
            controle_legalite = date_controle
    
    titre = ("Arrêté n°" + numero + " " + titre).strip()
    
    print("Identifiant final : ", identifiant)
    print("Titre final : ", titre)
    print("Date décision finale : ", date_decision)
    
    if (controle_legalite != ""):
        date_affichage = controle_legalite
    elif (date_decision != ""):
        date_affichage = date_decision
    else:
        date_affichage = date_publication("ok")
    
    dictionnaire = {"coll_nom": "Région Île-de-France",
                    "coll_siret": "23750007900312",
                    "identifiant": identifiant,
                    "numero": numero,
                    "titre": titre,
                    "type_acte": "AR",
                    "type": "Arrêtés de la Présidente",
                    "type_seance": "Présidente du conseil régional",
                    "theme": "Institution",
                    "date_controle": controle_legalite,
                    "date_decision": date_decision,
                    "date_publication": date_publication("ok"),
                    "date_affichage": date_affichage,
                    "texte_arrete": corps_visa_arrete,
                    "messages": str(messages),
                    "nom_fichier": nom_fichier,
                    "url1": url1
                   }
    
    return dictionnaire
    


# Declenchement

In [195]:

#chemin dans lequel se trouve les arretés: A modifier au besoin
os.chdir("C:/Users/utilisateur/Documents/Python Scripts/PUBLICATIONS ACTES/Arretes")
fileDirOrigin = "C:/Users/utilisateur/Documents/Python Scripts/PUBLICATIONS ACTES/Arretes"
fichier_a_inscrire = fileDirOrigin + '/extrait_arretes.csv'


In [196]:
liste_fichier =  [_ for _ in os.listdir(fileDirOrigin) if _.endswith('.pdf')]

# Ajouter une annotation de type timestamp pour la date de publication

In [197]:
for file in liste_fichier:
    if (file != "AR2023-034.pdf"):
        continue
    print("  === ", file, " ===  ")
    pdf = fitz.open(file)
    xref = None
    for index in range(0, pdf.page_count):
        print("Page numéro ", index+1, " :")
        page = pdf.load_page(index)
        for annot in page.annots():
            print("Annotation ", annot.xref, " : ")
            if ("FreeText" in annot.type):
                continue
            elif ("Stamp" in annot.type and page.number == 0):
                print("ICI ", annot.flags)
                print(annot.info)
                xref = annot.xref
                # page.add_stamp_annot(rect, stamp=10)
            else:
                print("Autre type : ", annot.type)
            print("---")
    if (xref is not None):
        page = pdf.load_page(0)
        annot = page.load_annot(xref)
        #stamp_text = annot.get_text()
        #regex_stamp_text = "Publié le"
        #stamp_text = re.sub(regex_stamp_text, regex_stamp_text + " " + datetime.today().strftime('%d/%m/%Y'), stamp_text)
        a,b,c,d = annot.rect
        # Rect(395.32000732421875, 75.0, 585.3200073242188, 87.0)
        #a = 395
        #b = 75
        #c = 585
        #d = 87
        rect = fitz.Rect(a,d+10,c,d+33)
        if (page.rotation % 360 == 90):
            rect = fitz.Rect(c+10,b,c+33,d)
        elif (page.rotation % 360 == 180):
            rect = fitz.Rect(a,b-33,c,b-10)
        elif (page.rotation % 360 == 270):
            rect = fitz.Rect(a-33,b,a-10,d)
        
        date_publication = datetime.today().strftime('%d/%m/%Y')
        date_publication = datetime(2023, 1, 20, 0, 15, 57, 779703).strftime('%d/%m/%Y')
        print("Rotation de la page : ", page.rotation)
        #page.set_rotation(0)
        annot = page.add_freetext_annot(rect, "Publication des actes de la Région Île-de-France\nPublié le " + date_publication, fontsize=8, fontname='helv', border_color=(0, 0, 0.5), text_color=(0, 0, 0.5), fill_color=(0.95, 0.95, 1), rotate=page.rotation, align=fitz.TEXT_ALIGN_LEFT)
        annot.set_border(width=1)
        annot.set_flags(fitz.PDF_ANNOT_IS_LOCKED|fitz.PDF_ANNOT_IS_READ_ONLY|fitz.PDF_ANNOT_IS_LOCKED_CONTENTS)
        annot.set_info(info={'name': 'Publication', 'title': 'Publication des actes administratifs', 'creationDate': str(datetime.today().strftime("D:%Y%m%d%H%M%S%z")), 'modDate': str(datetime.today().strftime("D:%Y%m%d%H%M%S%z")), 'subject': 'Date de publication'})
        annot.update()
        print(annot.info)
        pdf.save(pdf.name, incremental=True, encryption=fitz.PDF_ENCRYPT_KEEP, permissions=0)

In [198]:
# Traitement principal
liste_avant_identifiants = []
for fichier in liste_fichier:
    dictionnaire = traite_fichier_arrete(fichier, verbose=False)
    liste_avant_identifiants.append(dictionnaire)

 =====OCR===== 
DEBUT DOC AR2023-044.pdf
Texte replace / :  Envoyé en préfecture le 06022023
Texte replace / :  Reçu en préfecture le 06022023 ñ Publié le S L
Texte replace / :  ID : 075-237500079-20230203-2023 044-AR
Texte replace / :   
Texte replace / :  # Région fleceFrance Conseil régional
Texte replace / :  Ref : 123-CRIDF-00016
Texte replace / :  ARRETE N°2023-044 DU 3 FEVRIER 2023
Identifiant trouvé :  ARRETE N°2023-044 DU 3 FEVRIER 2023
Match :  {'numero': '2023-044 ', 'jour': '3', 'mois': 'FEVRIER', 'annee': '2023', 'titre': ''}
Identifiant part : AR2023-044
Date decision part : 2023-02-03
Identifiant part : 
Date decision part : 
Identifiant part : 
Date decision part : 
Identifiant final :  AR2023-044
Titre final :  Arrêté n°AR 2023-044 Portant délégations de signatures du pôle politiques sportives, de santé, de solidarité et de sécurité (PS4)
Date décision finale :  2023-02-03
 =====OCR===== 
DEBUT DOC AR2023-045.pdf
Texte replace / :  Envoyé en préfecture le 06022023 Reçu

In [203]:
#liste_avant_identifiants

In [200]:
field_names_csv = ['coll_nom','coll_siret','identifiant','numero','titre', 'type_acte', 'type', 'type_seance', 'theme',
                   'date_controle','date_decision',
                   'date_publication','date_affichage',
                   'texte_arrete','messages','nom_fichier',
                   'url1']


In [201]:

dictionnaire = {"coll_nom": "coll_nom",
                    "coll_siret": "coll_siret",
                    "identifiant": "identifiant",
                    "numero": "numero",
                    "titre": "titre",
                    "type_acte": "type_acte",
                    "type": "type",
                    "type_seance": "type_seance",
                    "theme": "theme",
                    "date_controle": "date_controle",
                    "date_decision": "date_decision",
                    "date_publication": "date_publication",
                    "date_affichage": "date_affichage",
                    "texte_arrete": "texte_arrete",
                    "messages": "messages",
                    "nom_fichier": "nom_fichier",
                    "url1": "url1"
                   }

append_dict_as_row(fichier_a_inscrire, dictionnaire, field_names_csv)

for dictionnaire in liste_avant_identifiants:
    append_dict_as_row(fichier_a_inscrire, dictionnaire, field_names_csv)


In [21]:
# liste_arrete

# Récupérer les délibs déjà publiées en open data

In [20]:
# URL API permettant de récupérer les délibérations déjà publiées
url_api_opendata_delibs = "https://data.iledefrance.fr/api/records/1.0/search/?dataset=actes-administratifs&q=&rows=10000&facet=type_acte&facet=type&facet=theme&facet=type_seance&refine.type_acte=AR&apikey=70b1e8b9be88b35fd29c3706bf258b3b6c9071ffa4e32504793e395b"

In [21]:
# Requête de l'API Opendatasoft
response_API = requests.get(url_api_opendata_delibs)
data = response_API.text
parse_json = json.loads(data)

In [22]:
# Nombre de délibérations trouvées: parse_json['nhits']

# Arbre des résultats: parse_json['records']

# Pour chaque entrée des résultats:
# Timestamp de l'entrée: parse_json['records']['record_timestamp']
# Accès aux champs de l'entée: parse_json['records']['fields']
# Champs identifiant de l'entée: parse_json['records']['fields']['identifiant']
# Champ date_publication de l'entée: parse_json['records']['fields']['date_publication']

dictionnaire_delibs_opendata = {}
for record in parse_json['records']:
    dictionnaire_delibs_opendata[record['fields']['identifiant']] = {
        'identifiant': record['fields']['identifiant'],
        'timestamp': record['record_timestamp'],
        'date_publication':record['fields']['date_publication']
    }

print("Code de réponse de l'API : ", response_API.status_code)
print("Nombre d'arrêtés déjà publiés en open data : ", parse_json['nhits'])
# print("Dictionnaire des délibérations publiées en open data : ", dictionnaire_delibs_opendata)


Code de réponse de l'API :  200
Nombre d'arrêtés déjà publiés en open data :  1377


In [23]:

# Lister les délibérations qui sont déjà publiées en open data et vont être exclues
liste_delibs_a_exclure = []
for dictionnaire in liste_avant_identifiants:
    if (dictionnaire['identifiant'] != ""):
        if (dictionnaire['identifiant'] in dictionnaire_delibs_opendata):
            print("Identifiant déjà publié en open data : ", dictionnaire['identifiant'], " ; date initiale de publication : ", dictionnaire_delibs_opendata[dictionnaire['identifiant']]['date_publication'])
            liste_delibs_a_exclure.append(dictionnaire['identifiant'])
        else:
            continue
            print("Délibération non encore publiée: ", dictionnaire['identifiant'])


